In [1]:
# Parameters
suggestion_file = "suggestion.csv"
optimization_file = "optimization.csv"


In [2]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.integrate import solve_ivp
# import pandas as pd

# # Input Variables
# T1Celsius = 120
# t1min = 17.5
# T2Celsius = 136  # Second temperature (Kelvin)
# t2min = 36.3  # Time at T2 (seconds)
# EquivalentsReagent1 = 1.3
# EquivalentsBASE1 = 2.5

# ConcentrationMolar = 0.82 #Concentration in molar, typically you would maximize the concentration if possible so this value could be reduced

# # #Variable bounds
# # T1CelsiusBounds = [20-200]
# # t1minBounds = [10-60]
# # T2CelsiusBounds= [20-200]
# # t2minBounds = [10-60]
# # EquivalentsReagent1Bounds = [1-2]
# # EquivalentsBASE1Bounds = [1-5]
# # ConcentrationMolarBounds = [0.1-1] #Concentration in molar

# # #PriceVariables
# # PriceSubstrate = 75 #€/kg
# # PriceBase = 4 #€/kg
# # PriceReagent = 20 #€/kg
# # PriceSolvent = 1 #€/kg
# # ReactorDepreciationYears = 5
# # base_reactor_price = 10000 #Reactor Costs are caculated by the formula: base_reactor_price+base_reactor_price*math.sqrt(ReactorSize)
# # YearlyProduction = 100 # t/a  densityAssumed To be one
# # #full installation costs are calculated by 10 times the full reactor cost

# # Read suggested parameters
# suggestion = pd.read_csv('suggestion.csv')

# # Input Variables from suggestion file
# T1Celsius = suggestion['T1Celsius'].iloc[0]
# t1min = suggestion['t1min'].iloc[0]
# T2Celsius = suggestion['T2Celsius'].iloc[0]
# t2min = suggestion['t2min'].iloc[0]
# EquivalentsReagent1 = suggestion['EquivalentsReagent1'].iloc[0]
# EquivalentsBASE1 = suggestion['EquivalentsBASE1'].iloc[0]
# ConcentrationMolar = suggestion['ConcentrationMolar'].iloc[0]
# print(ConcentrationMolar)
# # Constants
# R = 8.314e-3  # Gas constant in kJ/mol.K

# def reaction_rates(t, y, params, T):
#     """Calculates reaction rates."""
#     REAGENT1 = y[0]     #REAGENT1
#     BASE1 = y[1]     #BASE1
#     REAGENT1K = y[2]    #REAGENT1K
#     MEOH = y[3]     #MEOH
#     SUBSTRATEX = y[4]      #SUBSTRATEXO
#     SUBSTRATEXK = y[5]     #SUBSTRATEXK
#     PRODUCTXKK = y[6]    #PRODUCTXKK
#     PRODUCTXK = y[7]     #PRODUCTXK
#     IMP2 = y[8]     #IMP2
#     H2O = y[9]      #H2O

#     def calculate_k(k_ref, Ea, T_ref):  # Added T_ref
#         return k_ref * np.exp(-Ea / R * (1/T - 1/T_ref)) # Arrhenius equation with T_ref
    
#     def calculate_k_reverse(k_forward, Keq):
#             return k_forward / Keq

#     k1_f = calculate_k(params['k1_f'], params['Ea1_f'], params['T_ref'])
#     k2_f = calculate_k(params['k2_f'], params['Ea2_f'], params['T_ref'])
#     k3_f = calculate_k(params['k3_f'], params['Ea3_f'], params['T_ref'])  # Irreversible
#     k4_f = calculate_k(params['k4_f'], params['Ea4_f'], params['T_ref'])
#     k5_f = calculate_k(params['k5_f'], params['Ea5_f'], params['T_ref'])  # Irreversible

#     k1_r = calculate_k_reverse(k1_f, params['Keq1'])
#     k2_r = calculate_k_reverse(k2_f, params['Keq2'])
#     k4_r = calculate_k_reverse(k4_f, params['Keq4'])

#     r1 = k1_f * REAGENT1 * BASE1 - k1_r * REAGENT1K * MEOH
#     r2 = k2_f * SUBSTRATEX * BASE1 - k2_r * SUBSTRATEXK * MEOH
#     r3 = k3_f * REAGENT1K * SUBSTRATEXK  # Irreversible
#     r4 = k4_f * PRODUCTXKK * MEOH - k4_r * PRODUCTXK * BASE1
#     r5 = k5_f * PRODUCTXK * SUBSTRATEXK  # Irreversible

#     dREAGENT1dt = -r1
#     dBASE1dt = -r1 - r2 + r4
#     dREAGENT1Kdt = r1 - r3
#     dMEOHdt = r1 + r2 - r3
#     dSUBSTRATEXdt = -r2
#     dSUBSTRATEXKdt = r2 - r3 - r5
#     dPRODUCTXKKdt = r3 - r4
#     dPRODUCTXKdt = r4 - r5
#     dIMP2dt = r5
#     dH2Odt = r5

#     dydt = [dREAGENT1dt, dBASE1dt, dREAGENT1Kdt, dMEOHdt, dSUBSTRATEXdt, dSUBSTRATEXKdt, dPRODUCTXKKdt, dPRODUCTXKdt, dIMP2dt, dH2Odt]
#     return dydt

# def run_reaction(T1, t1, T2, t2, SUBSTRATEXK, BASE1, Reagent1, MeOH, V, params):
#     """Simulates the reaction with temperature changes."""
#     y0 = [Reagent1/V, BASE1/V, 0, MeOH/V, 0, SUBSTRATEXK/V, 0, 0, 0, 0]  # Initial concentrations

#     t_span1 = (0, t1)
#     t_span2 = (t1, t1 + t2)

#     sol1 = solve_ivp(reaction_rates, t_span1, y0, args=(params, T1), dense_output=True, max_step=1)
#     if sol1.status != 0:
#         raise RuntimeError("solve_ivp failed in the first step")
#     y0_updated = sol1.y[:, -1]

#     sol2 = solve_ivp(reaction_rates, t_span2, y0_updated, args=(params, T2), dense_output=True, max_step=1)
#     if sol2.status != 0:
#         raise RuntimeError("solve_ivp failed in the second step")

#     t_combined = np.concatenate([sol1.t, sol2.t])
#     y_combined = np.concatenate([sol1.y, sol2.y], axis=1)

#     sol = type('obj', (object,), {'t': t_combined, 'y': y_combined})()
#     return sol

# params = {
#     'k1_f': 0.0131867209533307, 'Ea1_f': 39.247, 'Keq1': 0.34305, 'Ea1_r': 35.0979,
#     'k2_f': 0, 'Ea2_f': 60.001, 'Keq2': 0.7140, 'Ea2_r': 59.999,
#     'k3_f': 0.0717498202, 'Ea3_f': 51.49215,  # Irreversible
#     'k4_f': 0.0000000177, 'Ea4_f': 61.37373, 'Keq4': 0.00004, 'Ea4_r': 0, #slightly modified
#     'k5_f': 0.001715, 'Ea5_f': 107.16032,  # Irreversible
#     'T_ref': 395.15 # Reference Temperature (Kelvin) - Add this to params
# }

# #Convert Units to units that are used in the model
# V = 1  # Volume (Liters)
# t1 = t1min*60  # Time at T1 (seconds)
# t2 = t2min*60  # Time at T1 (seconds)
# T1 = 273.15 + T1Celsius  # Initial temperature (Kelvin)
# T2 = T2Celsius + 273.15 # Second temperature (Kelvin)
# SUBSTRATEXK = ConcentrationMolar  # Initial moles of A
# Reagent1 = ConcentrationMolar*EquivalentsReagent1  # Initial moles of C (Reactant 5)
# BASE1 = ConcentrationMolar*EquivalentsBASE1  # Initial moles of B
# MeOH = 0.0161*1000  # Initial moles of solvent, assumed to be constant which is okay given it is the solvent)

# sol = run_reaction(T1, t1, T2, t2, SUBSTRATEXK, BASE1, Reagent1, MeOH, V, params)
# time = sol.t
# concentrations = sol.y
# ReactionTimeMin = t2min + t1min

# #Output parameters
# ReactionTime = t1 + t2
# Conversion = (concentrations[6, -1]+concentrations[7, -1])/SUBSTRATEXK
# Yield = Conversion
# Impurity = (concentrations[8, -1])/(SUBSTRATEXK/V)*100
# ImpurityXRatio = (concentrations[6, -1]+concentrations[7, -1]+concentrations[0, -1]+concentrations[2, -1])/(concentrations[6, -1]+concentrations[7, -1])
# print("Reaction Time = " + str(ReactionTimeMin) +" min")
# print("Yield = " + str(Conversion) + " %")
# print("Impurity1 Ratio = " + str(ImpurityXRatio))
# print("Impurity2 = " + str(Impurity) + " %")



In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd

print("Starting Virtual Lab Simulation...")

# Read the suggested parameters
print("Reading parameters from suggestion.csv...")
suggestion = pd.read_csv('suggestion.csv')

# Input Variables - read from suggestion file
T1Celsius = float(suggestion['T1Celsius'].iloc[0])
t1min = float(suggestion['t1min'].iloc[0])
T2Celsius = float(suggestion['T2Celsius'].iloc[0])
t2min = float(suggestion['t2min'].iloc[0])
EquivalentsReagent1 = float(suggestion['EquivalentsReagent1'].iloc[0])
EquivalentsBASE1 = float(suggestion['EquivalentsBASE1'].iloc[0])
ConcentrationMolar = float(suggestion['ConcentrationMolar'].iloc[0])
# ConcentrationMolar =  0.82

print("\nUsing parameters:")
print(f"T1Celsius = {T1Celsius}")
print(f"t1min = {t1min}")
print(f"T2Celsius = {T2Celsius}")
print(f"t2min = {t2min}")
print(f"EquivalentsReagent1 = {EquivalentsReagent1}")
print(f"EquivalentsBASE1 = {EquivalentsBASE1}")
print(f"ConcentrationMolar = {ConcentrationMolar}")

# Constants
R = 8.314e-3  # Gas constant in kJ/mol.K

def reaction_rates(t, y, params, T):
    """Calculates reaction rates."""
    REAGENT1 = y[0]     #REAGENT1
    BASE1 = y[1]     #BASE1
    REAGENT1K = y[2]    #REAGENT1K
    MEOH = y[3]     #MEOH
    SUBSTRATEX = y[4]      #SUBSTRATEXO
    SUBSTRATEXK = y[5]     #SUBSTRATEXK
    PRODUCTXKK = y[6]    #PRODUCTXKK
    PRODUCTXK = y[7]     #PRODUCTXK
    IMP2 = y[8]     #IMP2
    H2O = y[9]      #H2O

    def calculate_k(k_ref, Ea, T_ref):  # Added T_ref
        return k_ref * np.exp(-Ea / R * (1/T - 1/T_ref)) # Arrhenius equation with T_ref
    
    def calculate_k_reverse(k_forward, Keq):
        return k_forward / Keq

    k1_f = calculate_k(params['k1_f'], params['Ea1_f'], params['T_ref'])
    k2_f = calculate_k(params['k2_f'], params['Ea2_f'], params['T_ref'])
    k3_f = calculate_k(params['k3_f'], params['Ea3_f'], params['T_ref'])  # Irreversible
    k4_f = calculate_k(params['k4_f'], params['Ea4_f'], params['T_ref'])
    k5_f = calculate_k(params['k5_f'], params['Ea5_f'], params['T_ref'])  # Irreversible

    k1_r = calculate_k_reverse(k1_f, params['Keq1'])
    k2_r = calculate_k_reverse(k2_f, params['Keq2'])
    k4_r = calculate_k_reverse(k4_f, params['Keq4'])

    r1 = k1_f * REAGENT1 * BASE1 - k1_r * REAGENT1K * MEOH
    r2 = k2_f * SUBSTRATEX * BASE1 - k2_r * SUBSTRATEXK * MEOH
    r3 = k3_f * REAGENT1K * SUBSTRATEXK  # Irreversible
    r4 = k4_f * PRODUCTXKK * MEOH - k4_r * PRODUCTXK * BASE1
    r5 = k5_f * PRODUCTXK * SUBSTRATEXK  # Irreversible

    dREAGENT1dt = -r1
    dBASE1dt = -r1 - r2 + r4
    dREAGENT1Kdt = r1 - r3
    dMEOHdt = r1 + r2 - r3
    dSUBSTRATEXdt = -r2
    dSUBSTRATEXKdt = r2 - r3 - r5
    dPRODUCTXKKdt = r3 - r4
    dPRODUCTXKdt = r4 - r5
    dIMP2dt = r5
    dH2Odt = r5

    dydt = [dREAGENT1dt, dBASE1dt, dREAGENT1Kdt, dMEOHdt, dSUBSTRATEXdt, dSUBSTRATEXKdt, dPRODUCTXKKdt, dPRODUCTXKdt, dIMP2dt, dH2Odt]
    return dydt

def run_reaction(T1, t1, T2, t2, SUBSTRATEXK, BASE1, Reagent1, MeOH, V, params):
    """Simulates the reaction with temperature changes."""
    y0 = [Reagent1/V, BASE1/V, 0, MeOH/V, 0, SUBSTRATEXK/V, 0, 0, 0, 0]  # Initial concentrations

    t_span1 = (0, t1)
    t_span2 = (t1, t1 + t2)

    sol1 = solve_ivp(reaction_rates, t_span1, y0, args=(params, T1), dense_output=True, max_step=1)
    if sol1.status != 0:
        raise RuntimeError("solve_ivp failed in the first step")
    y0_updated = sol1.y[:, -1]

    sol2 = solve_ivp(reaction_rates, t_span2, y0_updated, args=(params, T2), dense_output=True, max_step=1)
    if sol2.status != 0:
        raise RuntimeError("solve_ivp failed in the second step")

    t_combined = np.concatenate([sol1.t, sol2.t])
    y_combined = np.concatenate([sol1.y, sol2.y], axis=1)

    sol = type('obj', (object,), {'t': t_combined, 'y': y_combined})()
    return sol

params = {
    'k1_f': 0.0131867209533307, 'Ea1_f': 39.247, 'Keq1': 0.34305, 'Ea1_r': 35.0979,
    'k2_f': 0, 'Ea2_f': 60.001, 'Keq2': 0.7140, 'Ea2_r': 59.999,
    'k3_f': 0.0717498202, 'Ea3_f': 51.49215,  # Irreversible
    'k4_f': 0.0000000177, 'Ea4_f': 61.37373, 'Keq4': 0.00004, 'Ea4_r': 0,
    'k5_f': 0.001715, 'Ea5_f': 107.16032,  # Irreversible
    'T_ref': 395.15 # Reference Temperature (Kelvin)
}

print("\nConverting units for simulation...")

#Convert Units to units that are used in the model
V = 1  # Volume (Liters)
t1 = t1min*60  # Time at T1 (seconds)
t2 = t2min*60  # Time at T2 (seconds)
T1 = 273.15 + T1Celsius  # Initial temperature (Kelvin)
T2 = T2Celsius + 273.15  # Second temperature (Kelvin)
SUBSTRATEXK = ConcentrationMolar  # Initial moles of A
Reagent1 = ConcentrationMolar*EquivalentsReagent1  # Initial moles of C
BASE1 = ConcentrationMolar*EquivalentsBASE1  # Initial moles of B
MeOH = 0.0161*1000  # Initial moles of solvent

print("\nRunning reaction simulation...")
sol = run_reaction(T1, t1, T2, t2, SUBSTRATEXK, BASE1, Reagent1, MeOH, V, params)
time = sol.t
concentrations = sol.y
ReactionTimeMin = t2min + t1min

#Calculate output parameters
ReactionTime = t1 + t2
Conversion = (concentrations[6, -1]+concentrations[7, -1])/SUBSTRATEXK
Yield = Conversion
Impurity = (concentrations[8, -1])/(SUBSTRATEXK/V)*100
ImpurityXRatio = (concentrations[6, -1]+concentrations[7, -1]+concentrations[0, -1]+concentrations[2, -1])/(concentrations[6, -1]+concentrations[7, -1])

print("\nSimulation Results:")
print(f"Reaction Time = {ReactionTimeMin:.2f} min")
print(f"Yield = {Conversion:.6f}")
print(f"Impurity1 Ratio = {ImpurityXRatio:.6f}")
print(f"Impurity2 = {Impurity:.6f} %")

# Create results DataFrame with the ACTUAL parameters used
results_df = pd.DataFrame({
    'T1Celsius': [T1Celsius],
    't1min': [t1min],
    'T2Celsius': [T2Celsius],
    't2min': [t2min],
    'EquivalentsReagent1': [EquivalentsReagent1],
    'EquivalentsBASE1': [EquivalentsBASE1],
    'ConcentrationMolar': [ConcentrationMolar],
    'Yield': [Conversion],
    'Impurity': [Impurity],
    'ImpurityXRatio': [ImpurityXRatio]
})

# Save results
print("\nSaving results to optimization.csv...")
results_df.to_csv('optimization.csv', index=False)
print("Results saved successfully.")

Starting Virtual Lab Simulation...
Reading parameters from suggestion.csv...

Using parameters:
T1Celsius = 122.020696246808
t1min = 36.60227127394168
T2Celsius = 141.8091143197875
t2min = 35.768025994916464
EquivalentsReagent1 = 1.9980580750284005
EquivalentsBASE1 = 1.959526877288282
ConcentrationMolar = 0.82

Converting units for simulation...

Running reaction simulation...



Simulation Results:
Reaction Time = 72.37 min
Yield = 0.999267
Impurity1 Ratio = 1.999417
Impurity2 = 0.010725 %

Saving results to optimization.csv...
Results saved successfully.


In [4]:
# import random

# def add_noise(value, noise):
#     """Adds ±noise% random noise to the given value."""
#     noise_factor = random.uniform(-noise*0.01, 0.01*noise)  # Random value between -1% and 1%
#     return value * (1 + noise_factor)

# ConversionNoise = add_noise(Conversion,1)
# ImpurityXNoise = add_noise(ImpurityXRatio,1)
# ImpurityNoise = add_noise(Impurity,10)

In [5]:
# import math

# def calculate_weight_percentages(chemicals):
#     """Calculates weight percentages based on molarity and molar mass."""
#     total_mass = sum(c["molarity"] * c["molar_mass"] for c in chemicals.values())
#     weight_percentages = {name: (c["molarity"] * c["molar_mass"] / total_mass) for name, c in chemicals.items()}
#     return weight_percentages


# def estimate_reactor_process_cost(chemicals, ReactorSize, Yield, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price):
#     #Determine reactor costs
#     reactorCost = base_reactor_price+base_reactor_price*math.sqrt(ReactorSize)

#     #FullInstallationCost
#     FullInstallationCost = reactorCost*10    #simplified

#     #Reactor Depreciation over 5 years
#     yearly_depreciation = reactorCost/ReactorDepreciationYears
#     Reactorcost_per_kg = yearly_depreciation/(YearlyProduction*1000)

#     return Reactorcost_per_kg 

# def estimate_chemical_process_cost(chemicals, ReactorSize, Yield, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price):
#     total_chemical_cost = 0

#     for chem, props in chemicals.items():
#         molarity = props["molarity"]  # mol/L
#         molar_mass = props["molar_mass"]  # g/mol
#         cost_per_kg = props["cost_per_kg"]  # cost per kg
#         equivalents = props.get("equivalents", 1)  # Default to 1 if not provided
        
#         # Convert molarity to kg needed per 1 kg of product
#         kg_needed = (equivalents * molar_mass / (chemicals["substrate"]["molar_mass"]+chemicals["reagent"]["molar_mass"])) / Yield
        
#         # Calculate cost for this chemical
#         cost = kg_needed * cost_per_kg
#         total_chemical_cost += cost
#     return total_chemical_cost 


# #Density of the mixture
# densityMixture = 0.820

# #chemicals = {"solvent": 6, "M": 14.5, "K": 50, "O": 3500}  # Costs in euro per kg - based on sigma price per kg/10
# chemicals = {
#     "solvent": {"molarity": densityMixture*1000/32, "molar_mass": 32, "cost_per_kg": PriceSolvent, "equivalents": 1/ConcentrationMolar/160},
#     "reagent": {"molarity": EquivalentsReagent1*ConcentrationMolar, "molar_mass": 150, "cost_per_kg": PriceReagent, "equivalents": EquivalentsReagent1},
#     "substrate": {"molarity":ConcentrationMolar, "molar_mass": 160, "cost_per_kg": PriceSubstrate, "equivalents": 1},
#     "Base": {"molarity":EquivalentsBASE1*ConcentrationMolar, "molar_mass": 50, "cost_per_kg": PriceBase, "equivalents": EquivalentsBASE1},
# }


# #ConcentrationProductInProcess = 10 # wt%
# m_substrate_per_L = ConcentrationMolar*chemicals["substrate"]["molar_mass"]*0.001
# L_per_kg_made = 1/m_substrate_per_L
# RequiredFlowRate = YearlyProduction *1000 /356/24/60 *  L_per_kg_made  #L/min
# #print("Flow rate = "+ str(RequiredFlowRate) +" L/min")
# ReactorSize = RequiredFlowRate*ReactionTime/60 #L/min
# #print("Reactorsize = " + str(ReactorSize) + " L")

# ChemicalCostPerKg = estimate_chemical_process_cost(chemicals, ReactorSize, Yield, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price)
# ReactorcostPerKg = estimate_reactor_process_cost(chemicals, ReactorSize, Yield, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price)
# pricePerkg = ReactorcostPerKg + ChemicalCostPerKg

# ChemicalCostPerKgNoise = estimate_chemical_process_cost(chemicals, ReactorSize, ConversionNoise, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price)
# ReactorcostPerKgNoise = estimate_reactor_process_cost(chemicals, ReactorSize, ConversionNoise, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price)
# pricePerkgNoise = ReactorcostPerKgNoise + ChemicalCostPerKgNoise
# print("Yield = " + str(Conversion) + " %")
# print("Impurity1 Ratio = " + str(ImpurityXRatio))
# print("Impurity2 = " + str(Impurity) + " %")
# print("Price = " + str(pricePerkg) + " €/kg")



In [6]:
# print("Yield with noise = " + str(ConversionNoise))
# print("Impurity1 Ratio with noise = " + str(ImpurityXRatio))
# print("Impurity2 with noise = " + str(ImpurityNoise))
# ChemicalCostPerKgNoise = estimate_chemical_process_cost(chemicals, ReactorSize, ConversionNoise, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price)
# ReactorcostPerKgNoise = estimate_reactor_process_cost(chemicals, ReactorSize, ConversionNoise, YearlyProduction,RequiredFlowRate, ReactorDepreciationYears, base_reactor_price)
# pricePerkgNoise = ReactorcostPerKgNoise + ChemicalCostPerKgNoise
# print("price Per kg with Noise = " + str(pricePerkgNoise) + " €/kg")

In [7]:
# # Plotting
# plt.figure(figsize=(10, 6),dpi = 300)  # Adjust figure size for better visualization

# labels = ['Reagent1', 'Base', 'Reagent', 'MeOH', 'SubstrateX', 'SubstratexK', 'ProductK', 'ProductXK', 'IMP2', 'H2O']  # Labels for plotting

# for i in range(concentrations.shape[0]):
#     plt.plot(time, concentrations[i, :], label=labels[i])

# plt.xlabel('Time (s)')
# plt.ylabel('Concentration (mol/L)')
# plt.title('Concentration Profiles')
# plt.legend(loc='best')  # Place legend in the best location
# plt.grid(True)
# plt.tight_layout()  # Adjust layout to prevent labels from overlapping
# plt.show()

# plt.figure(figsize=(10, 6),dpi = 300)  # Adjust figure size for better visualization

# labels = ['Reagent1', 'Base', 'Reagent', 'MeOH', 'SubstrateX', 'SubstratexK', 'ProductK', 'ProductXK', 'IMP2', 'H2O']  # Labels for plotting

# plt.plot(time, concentrations[0, :], label=labels[0])
# plt.plot(time, concentrations[5, :], label=labels[5])
# plt.plot(time, concentrations[6, :], label=labels[6])
# plt.plot(time, concentrations[7, :], label=labels[7])
# plt.plot(time, concentrations[8, :], label=labels[8])

# plt.xlabel('Time (s)')
# plt.ylabel('Concentration (mol/L)')
# plt.title('Concentration Profiles')
# plt.legend(loc='best')  # Place legend in the best location
# plt.grid(True)
# plt.tight_layout()  # Adjust layout to prevent labels from overlapping
# plt.show()

# plt.figure(figsize=(10, 6),dpi = 300)  # Adjust figure size for better visualization

# labels = ['Reagent1', 'Base', 'Reagent', 'MeOH', 'SubstrateX', 'SubstratexK', 'ProductK', 'ProductXK', 'IMP2', 'H2O']  # Labels for plotting
# plt.plot(time, (concentrations[6, :]+concentrations[7, :])/(SUBSTRATEXK/V)*100, label="Conversion")
# plt.plot(time, (concentrations[8, :])/(SUBSTRATEXK/V)*100, label="Impurity")
# plt.xlabel('Time (s)')
# plt.ylabel('Yield')
# plt.title('Yield')
# plt.legend(loc='best')  # Place legend in the best location
# plt.ylim(0,100)
# plt.grid(True)
# plt.tight_layout()  # Adjust layout to prevent labels from overlapping
# plt.show()

